In [1]:
# default_exp grids

In [2]:
# hide
# no_test
! [ -e /content ] && pip install -Uqq git+https://github.com/thinkingmachines/geowrangler.git

In [3]:
# hide
# no_test
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
# hide
# no_test
# conditionally load nb_black cell magic
import sys

if "google.colab" not in sys.modules:
    from IPython import get_ipython

    ipython = get_ipython()
    ipython.magic("reload_ext lab_black")

# Grids

> grid generators


[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thinkingmachines/geowrangler/blob/master/notebooks/00_grids.ipynb)


In [25]:
# exporti
import logging
from typing import List, Tuple, Union

import numpy as np
from fastcore.basics import patch
from geopandas import GeoDataFrame
from pyproj import Transformer
from shapely.geometry import Polygon

logger = logging.getLogger(__name__)

In [26]:
# export


class SquareGridBoundary:
    """Reusing Boundary. x_min, y_min, x_max, and y_max are in the the target crs"""

    def __init__(self, x_min: float, y_min: float, x_max: float, y_max: float):
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max

    def get_range_subset(
        self, x_min: float, y_min: float, x_max: float, y_max: float, cell_size: float
    ) -> Tuple[float, List[float], float, List[float]]:
        """Returns a subset of grids from the orginal boundary based on the boundary and a grid size"""
        xrange = np.arange(self.x_min, self.x_max, cell_size)
        yrange = np.arange(self.y_min, self.y_max, cell_size)
        x_mask = (xrange >= x_min) & (xrange <= x_max)
        y_mask = (yrange >= y_min) & (yrange <= y_max)
        x_idx_offset = np.nonzero(x_mask)[0][0]
        y_idx_offset = np.nonzero(x_mask)[0][0]
        return (
            x_idx_offset,
            xrange[x_mask],
            y_idx_offset,
            yrange[y_mask],
        )

In [13]:
# export


class SquareGridGenerator:
    def __init__(
        self,
        gdf: GeoDataFrame,  # a geodataframe to create grids from
        cell_size: float,  # height and width of a square cell in meters
        grid_projection: str = "EPSG:3857",  # projection of grid output
        boundary: Union[SquareGridBoundary, List[float]] = None,  # original boundary
    ):
        self.gdf = gdf
        self.cell_size = cell_size
        self.grid_projection = grid_projection
        self.boundary = boundary

In [14]:
# export


@patch
def create_cell(
    self: SquareGridGenerator,
    x: float,  # x coord of top left
    y: float,  # y coord of top left
) -> Polygon:
    """Create a square cell based on the top left coordinates and cell_size"""
    return Polygon(
        [
            (x, y),
            (x + self.cell_size, y),
            (x + self.cell_size, y + self.cell_size),
            (x, y + self.cell_size),
        ]
    )

In [22]:
# export


@patch
def generate_grid(self: SquareGridGenerator) -> GeoDataFrame:
    polygons = []
    reprojected_gdf = self.gdf.to_crs(self.grid_projection)
    if self.boundary is None:
        boundary = SquareGridBoundary(*reprojected_gdf.total_bounds)
    elif isinstance(self.boundary, SquareGridBoundary):
        boundary = self.boundary
    else:
        transformer = Transformer.from_crs(
            self.gdf.crs, reprojected_gdf.crs, always_xy=True
        )
        x_min, y_min = transformer.transform(self.boundary[0], self.boundary[1])
        x_max, y_max = transformer.transform(self.boundary[2], self.boundary[3])
        boundary = SquareGridBoundary(x_min, y_min, x_max, y_max)

    # TODO: Catch case where no geometries are within the boundary
    x_idx_offset, xrange, y_idx_offset, yrange = boundary.get_range_subset(
        *reprojected_gdf.total_bounds, cell_size=self.cell_size
    )
    for x_idx, x in enumerate(xrange):
        for y_idx, y in enumerate(yrange):
            polygons.append(
                {
                    "x": x_idx + x_idx_offset,
                    "y": y_idx + y_idx_offset,
                    "geometry": self.create_cell(x, y),
                }
            )

    dest = GeoDataFrame(polygons, geometry="geometry", crs=self.grid_projection)
    dest_reproject = dest.to_crs(self.gdf.crs)
    final = dest_reproject[dest_reproject.intersects(self.gdf.unary_union)]
    return final

In [29]:
# hide
# no_test
from nbdev.export import notebook2script

notebook2script("00_grids.ipynb")

Converted 00_grids.ipynb.
